In [1]:
import pandas as pd
from nominatim import coordinates_dataframe

# Importing

In [2]:
# Read data
df = pd.read_csv("apartments_data_scraped.csv", index_col=0)

# Copy for future comparison 
df_original = df.copy()

## Disclaimer on API calls

In [3]:
# Call function to get the coordinates of each street via API
'''
!!! ATTENTION !!!

The nominatim API has a limit rate of one call per second, so the result of the function is saved in a file called apartments_data.csv.

I already did that, so the function will be commented down bellow, if you need to rerun this file, uncomment the line. 

'''
# coordinates_dataframe(df)

'\n!!! ATTENTION !!!\n\nThe nominatim API has a limit rate of one call per second, so the result of the function is saved in a file called apartments_data.csv.\n\nI already did that, so the function will be commented down bellow, if you need to rerun this file, uncomment the line. \n\n'

In [4]:
# Get dataframe with coords
df = pd.read_csv('apartments_data.csv')

# Initial exploration

In [5]:
# First insights on data
print(df.head(10))
df.info()

   Unnamed: 0.1  Unnamed: 0                   neighborhood  \
0             0           0    Parque Residencial Aquarius   
1             1           1                  Jardim Fátima   
2             2           2               Jardim Esplanada   
3             3           3          Palmeiras de São José   
4             4           4            Jardim Oswaldo Cruz   
5             5           5  Parque Residencial Flamboyant   
6             6           6                       Vila Ema   
7             7           7              Parque Industrial   
8             8           8            Jardim Oswaldo Cruz   
9             9           9          Palmeiras de São José   

                   city                        street  \
0   São José dos Campos  Rua Benedito Osvaldo Lecques   
1   São José dos Campos                  Rua Ipiranga   
2   São José dos Campos           Rua Laurent Martins   
3   São José dos Campos                Rua José Cobra   
4   São José dos Campos       Ru

In [6]:
# Drop unnamed columns
df = df.drop(columns=['Unnamed: 0.1',  'Unnamed: 0'])
print(df.head())

                  neighborhood                  city  \
0  Parque Residencial Aquarius   São José dos Campos   
1                Jardim Fátima   São José dos Campos   
2             Jardim Esplanada   São José dos Campos   
3        Palmeiras de São José   São José dos Campos   
4          Jardim Oswaldo Cruz   São José dos Campos   

                         street  \
0  Rua Benedito Osvaldo Lecques   
1                  Rua Ipiranga   
2           Rua Laurent Martins   
3                Rua José Cobra   
4       Rua República do Líbano   

                                         description area rooms parking  \
0  Cobertura com 3 dormitórios à venda, 193 m² po...  160     3       3   
1  Casa de Campo, Piscina Adulto e Infantil, Chur...   87     2       2   
2  Em andar alto e com uma vista incrível para a ...  124     4       3   
3  Descrição do Imóvel:\nApartamento mobiliado de...   47     2     NaN   
4  104 M² - 3 DORMITÓRIOS / 1 SUÍTE 2 VAGAS PARAL...  104     3       2   

 

## Getting columns for PCA

There're 07 columns that we expected to be numerical, and so we can use in our PCA analysis:
- Area
- Rooms
- Parking
- Bathrooms
- Lat
- Lon
- Price

Only lat and lon are presented as numerical in cel #19. Let's see what's happening in the other columns

In [7]:
# Get the numerical variables for a PCA test
df = df[['area', 'rooms', 'parking', 'bathrooms', 'lat', 'lon', 'price']]

## Evaluating Area

In [8]:
# Create a "is_numeric" column for each column
columns = df.columns

for column in columns:
    
    column_name = column + '_is_numeric'
    df[column_name] = pd.to_numeric(df[column], errors='coerce')

print(df.head(10))

      area  rooms parking bathrooms        lat        lon               price  \
0      160      3       3         2 -23.222113 -45.909534                 NaN   
1       87      2       2         3 -23.207893 -45.896603  Valor sob consulta   
2      124      4       3         3 -23.197917 -45.911362             1090000   
3       47      2     NaN         2 -23.250217 -45.917066              335000   
4      104      3       2         3 -23.201250 -45.883484              590000   
5       60      2       1         2 -23.214223 -45.851209              300000   
6       80      2       2         2 -23.203881 -45.902105              848500   
7       65      2       1         2 -23.226731 -45.903681              515000   
8  56 - 73  2 - 3   1 - 2         2 -23.200832 -45.883038              530000   
9       66      3       2         2        NaN        NaN              520000   

   area_is_numeric  rooms_is_numeric  parking_is_numeric  \
0            160.0               3.0            

In [9]:
# Analyzing area column
non_numeric_area = df[df['area_is_numeric'].isna()]
non_numeric_area = non_numeric_area[['area', 'area_is_numeric']]
number_non_numeric_area = len(non_numeric_area)
print('Number of non-numeric areas: ', number_non_numeric_area)
print(non_numeric_area.head(10))

Number of non-numeric areas:  28
             area  area_is_numeric
8         56 - 73              NaN
16        38 - 62              NaN
23        56 - 73              NaN
31        56 - 73              NaN
41        58 - 60              NaN
43        52 - 55              NaN
73        51 - 52              NaN
74        65 - 70              NaN
85   50 - 58 - 63              NaN
156       61 - 62              NaN


28 apartments have more than one area separated by "-", this means the price should vary between each area. Since we only have the price for one area, and we don't know wich one it represents, we will exclude these apartments from our analysis  


In [10]:
mask = ~df['area_is_numeric'].isna()
df = df[mask]
print(df.head(10))

   area rooms parking bathrooms        lat        lon               price  \
0   160     3       3         2 -23.222113 -45.909534                 NaN   
1    87     2       2         3 -23.207893 -45.896603  Valor sob consulta   
2   124     4       3         3 -23.197917 -45.911362             1090000   
3    47     2     NaN         2 -23.250217 -45.917066              335000   
4   104     3       2         3 -23.201250 -45.883484              590000   
5    60     2       1         2 -23.214223 -45.851209              300000   
6    80     2       2         2 -23.203881 -45.902105              848500   
7    65     2       1         2 -23.226731 -45.903681              515000   
9    66     3       2         2        NaN        NaN              520000   
10   55     2       1         2        NaN        NaN              359900   

    area_is_numeric  rooms_is_numeric  parking_is_numeric  \
0             160.0               3.0                 3.0   
1              87.0           

## Evaluating Rooms

In [11]:
# Analyzing rooms column
non_numeric_rooms = df[df['rooms_is_numeric'].isna()]
non_numeric_rooms = non_numeric_rooms[['rooms', 'rooms_is_numeric']]
number_non_numeric_rooms = len(non_numeric_rooms)
print('Number of non-numeric rooms: ', number_non_numeric_rooms)
print(non_numeric_rooms.head(10))

Number of non-numeric rooms:  8
            rooms  rooms_is_numeric
46          2 - 3               NaN
205         2 - 3               NaN
372         3 - 4               NaN
490         2 - 3               NaN
597   3 - 10 - 11               NaN
1086        4 - 5               NaN
1282        2 - 3               NaN
1368        4 - 5               NaN


There're 8 rows with rooms that vary and we don't know wich one represents the price of the apartment. Thus, we will also exclude those datas from our analysis

In [12]:
mask = ~df['rooms_is_numeric'].isna()
df = df[mask]
print(df.head(10))

   area rooms parking bathrooms        lat        lon               price  \
0   160     3       3         2 -23.222113 -45.909534                 NaN   
1    87     2       2         3 -23.207893 -45.896603  Valor sob consulta   
2   124     4       3         3 -23.197917 -45.911362             1090000   
3    47     2     NaN         2 -23.250217 -45.917066              335000   
4   104     3       2         3 -23.201250 -45.883484              590000   
5    60     2       1         2 -23.214223 -45.851209              300000   
6    80     2       2         2 -23.203881 -45.902105              848500   
7    65     2       1         2 -23.226731 -45.903681              515000   
9    66     3       2         2        NaN        NaN              520000   
10   55     2       1         2        NaN        NaN              359900   

    area_is_numeric  rooms_is_numeric  parking_is_numeric  \
0             160.0               3.0                 3.0   
1              87.0           

## Evaluating Parking

In [13]:
# Analyzing parking column
non_numeric_parking = df[df['parking_is_numeric'].isna()]
non_numeric_parking = non_numeric_parking[['parking', 'parking_is_numeric']]
number_non_numeric_parking = len(non_numeric_parking)
print('Number of non-numeric parking: ', number_non_numeric_parking)
print(non_numeric_parking.head(10))

Number of non-numeric parking:  46
    parking  parking_is_numeric
3       NaN                 NaN
35    1 - 2                 NaN
72      NaN                 NaN
130     NaN                 NaN
139   1 - 2                 NaN
239     NaN                 NaN
304   2 - 7                 NaN
318     NaN                 NaN
325     NaN                 NaN
375     NaN                 NaN


46 apartments either have no information about parking or do not have any information at all. For that, we will also exclude them from our analisys

In [14]:
mask = ~df['parking_is_numeric'].isna()
df = df[mask]
print(df.head(10))

   area rooms parking bathrooms        lat        lon               price  \
0   160     3       3         2 -23.222113 -45.909534                 NaN   
1    87     2       2         3 -23.207893 -45.896603  Valor sob consulta   
2   124     4       3         3 -23.197917 -45.911362             1090000   
4   104     3       2         3 -23.201250 -45.883484              590000   
5    60     2       1         2 -23.214223 -45.851209              300000   
6    80     2       2         2 -23.203881 -45.902105              848500   
7    65     2       1         2 -23.226731 -45.903681              515000   
9    66     3       2         2        NaN        NaN              520000   
10   55     2       1         2        NaN        NaN              359900   
11   42     2       1         2 -23.231972 -45.894292              300000   

    area_is_numeric  rooms_is_numeric  parking_is_numeric  \
0             160.0               3.0                 3.0   
1              87.0           

## Evaluating Bathrooms

In [15]:
# Analyzing bathroom column
non_numeric_bathrooms = df[df['bathrooms_is_numeric'].isna()]
non_numeric_bathrooms = non_numeric_bathrooms[['bathrooms', 'bathrooms_is_numeric']]
number_non_numeric_bathrooms = len(non_numeric_bathrooms)
print('Number of non-numeric bathrooms: ', number_non_numeric_bathrooms)
print(non_numeric_bathrooms.head(10))

Number of non-numeric bathrooms:  28
     bathrooms  bathrooms_is_numeric
14       2 - 3                   NaN
80       1 - 2                   NaN
146      1 - 2                   NaN
160      2 - 3                   NaN
162      1 - 2                   NaN
175  3 - 4 - 5                   NaN
207      2 - 3                   NaN
278      2 - 3                   NaN
289      1 - 2                   NaN
319      1 - 2                   NaN


28 apartments have ambiguos infromation about number of bathrooms, we will exclude them from the analysis

In [16]:
mask = ~df['bathrooms_is_numeric'].isna()
df = df[mask]
print(df.head(10))

   area rooms parking bathrooms        lat        lon               price  \
0   160     3       3         2 -23.222113 -45.909534                 NaN   
1    87     2       2         3 -23.207893 -45.896603  Valor sob consulta   
2   124     4       3         3 -23.197917 -45.911362             1090000   
4   104     3       2         3 -23.201250 -45.883484              590000   
5    60     2       1         2 -23.214223 -45.851209              300000   
6    80     2       2         2 -23.203881 -45.902105              848500   
7    65     2       1         2 -23.226731 -45.903681              515000   
9    66     3       2         2        NaN        NaN              520000   
10   55     2       1         2        NaN        NaN              359900   
11   42     2       1         2 -23.231972 -45.894292              300000   

    area_is_numeric  rooms_is_numeric  parking_is_numeric  \
0             160.0               3.0                 3.0   
1              87.0           

## Evaluating Price

In [17]:
# Analyzing price column
non_numeric_price = df[df['price_is_numeric'].isna()]
non_numeric_price = non_numeric_price[['price', 'price_is_numeric']]
number_non_numeric_price = len(non_numeric_price)
print('Number of non-numeric price: ', number_non_numeric_price)
print(non_numeric_price.head(10))

Number of non-numeric price:  148
                 price  price_is_numeric
0                  NaN               NaN
1   Valor sob consulta               NaN
12                 NaN               NaN
13                 NaN               NaN
33                 NaN               NaN
36                 NaN               NaN
38  Valor sob consulta               NaN
44                 NaN               NaN
59                 NaN               NaN
91                 NaN               NaN


We got 148 apartments with no price. They will be excluded from the evaluation since it's the most important variable tha we have.

In [18]:
mask = ~df['price_is_numeric'].isna()
df = df[mask]
print(df.head(10))

   area rooms parking bathrooms        lat        lon    price  \
2   124     4       3         3 -23.197917 -45.911362  1090000   
4   104     3       2         3 -23.201250 -45.883484   590000   
5    60     2       1         2 -23.214223 -45.851209   300000   
6    80     2       2         2 -23.203881 -45.902105   848500   
7    65     2       1         2 -23.226731 -45.903681   515000   
9    66     3       2         2        NaN        NaN   520000   
10   55     2       1         2        NaN        NaN   359900   
11   42     2       1         2 -23.231972 -45.894292   300000   
15  278     3       4         5 -23.208221 -45.908325  3500000   
17  233     4       3         5 -23.203424 -45.910521  2700000   

    area_is_numeric  rooms_is_numeric  parking_is_numeric  \
2             124.0               4.0                 3.0   
4             104.0               3.0                 2.0   
5              60.0               2.0                 1.0   
6              80.0          

## Evaluating Lat and Lon

In [19]:
# Exclude apartments without coordinates
non_numeric_coord = df[(df['lat'].isna()) | (df['lon'].isna())]
number_non_numeric_coords = len(non_numeric_coord)

df = df[~(df['lat'].isna()) | ~(df['lon'].isna())]
total_cleaned_observations = len(df)

print('Number of non-numeric coords: ', number_non_numeric_coords)
print('Complete observations : ', len(df))

Number of non-numeric coords:  366
Complete observations :  875


# Conclusion

In [20]:
total_observations = len(df_original)
print('Total observations: ', total_observations)
print('No Area: ', number_non_numeric_area, f'{round(number_non_numeric_area/total_observations*100,2)} %')
print('No Rooms: ', number_non_numeric_rooms, f'{round(number_non_numeric_rooms/total_observations*100,2)} %')
print('No Bathrooms: ', number_non_numeric_bathrooms, f'{round(number_non_numeric_bathrooms/total_observations*100,2)} %')
print('No Parking: ', number_non_numeric_parking, f'{round(number_non_numeric_parking/total_observations*100,2)} %')
print('No Coords: ', number_non_numeric_coords, f'{round(number_non_numeric_coords/total_observations*100,2)} %')
print('No Price: ', number_non_numeric_price, f'{round(number_non_numeric_price/total_observations*100,2)} %')
print('-------------------------------------------------------')
print('Remaining observations: ', len(df), f'{round(len(df)/total_observations,2)} %')

Total observations:  1499
No Area:  28 1.87 %
No Rooms:  8 0.53 %
No Bathrooms:  28 1.87 %
No Parking:  46 3.07 %
No Coords:  366 24.42 %
No Price:  148 9.87 %
-------------------------------------------------------
Remaining observations:  875 0.58 %


In [21]:
df = df[['area', 'rooms', 'parking', 'bathrooms', 'lat', 'lon', 'price']]
df.to_csv('apartments_data_cleaned.csv')